In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [8]:
# Initialize MLflow
mlflow.set_experiment("deployment model")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

Traceback (most recent call last):
  File "c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\store\tracking\file_store.py", line 368, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\store\tracking\file_store.py", line 466, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1636, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1629, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    r

🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/171386785805940664/runs/1be5764a1c4d4d16b9cb1adea8879d6b
🧪 View experiment at: http://localhost:5000/#/experiments/171386785805940664


2025/11/04 22:41:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 22:41:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/171386785805940664/runs/3d1ead9a9c2d4d4bb8dd13077e0d76f4
🧪 View experiment at: http://localhost:5000/#/experiments/171386785805940664


2025/11/04 22:41:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 22:41:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/171386785805940664/runs/ac6f95b6ed724e95a6eba7f839475844
🧪 View experiment at: http://localhost:5000/#/experiments/171386785805940664


2025/11/04 22:41:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 22:41:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/171386785805940664/runs/605dd77882c548e3859b3694f7822347
🧪 View experiment at: http://localhost:5000/#/experiments/171386785805940664


In [20]:
import mlflow

# Define run ID and model path
run_id = "1be5764a1c4d4d16b9cb1adea8879d6b"
model_uri = f"runs:/{run_id}/model"

# Register the model in the MLflow Model Registry
try:
    result = mlflow.register_model(model_uri=model_uri, name="Logistic_Regression")
    print(f"✅ Model registered successfully: {result.name}, version: {result.version}")
except Exception as e:
    print(f"❌ Error while registering model: {e}")



Successfully registered model 'Logistic_Regression'.
2025/11/04 23:01:59 WARNING mlflow.tracking._model_registry.fluent: Run with id 1be5764a1c4d4d16b9cb1adea8879d6b has no artifacts at artifact path 'model', registering model based on models:/m-4ae41275364347078495ca304eefba36 instead
2025/11/04 23:01:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic_Regression, version 1


✅ Model registered successfully: Logistic_Regression, version: 1


Created version '1' of model 'Logistic_Regression'.


In [15]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])